In [1]:
from data_wrangle import data_wrangle as dw
from datetime import datetime, date, timedelta
import time
import itertools
import pickle

# Basics
import numpy as np
import pandas as pd
import yfinance as yf 


#Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Optimization part
from cvxopt import matrix
from cvxopt.solvers import qp
from sklearn.covariance import LedoitWolf
from pypfopt import risk_models, expected_returns, plotting, EfficientFrontier

# sclearn RandomForest - Returns, ML Metrics and tools
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, GridSearchCV
from sklearn.pipeline import make_pipeline

# GARCH - Volatility
from arch import arch_model
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [2]:
# class model_validation:
    
#     def __init__(self, tickers, start, end, features_tickers):
#         self.tickers = tickers
#         self.start = start
#         self.end = end

#         self.features = yf.download(features_tickers, start=self.start, end=self.end, 
#                                     interval='1wk').fillna(method='ffill')
#         self.ohlc = yf.download(self.tickers, start=self.start, end=self.end, interval='1wk')
    
   
#     def tech_features(self):
#         inst = dw()
#         output = pd.DataFrame()
#         for i in self.ohlc.columns.levels[1]:

#             tech = inst.get_tech_features(close=self.ohlc['Adj Close'][i],
#                                               volume=self.ohlc['Volume'][i],
#                                               low=self.ohlc['Low'][i],
#                                               high=self.ohlc['High'][i],
#                                               concat=True).dropna()
#             tech.columns = pd.MultiIndex.from_product([[i], tech.columns])

#             output = pd.concat([output, tech], axis=1)
            
#         self.ohlc = self.ohlc.loc[output.index]
#         self.features = self.features['Adj Close'].loc[output.index]
#         self.tech = output
#         print('Done!')
        

#     def binary_classificator(self, x):
#         if x > 0:
#             return 1
#         else:
#             return -1
        
    
#     def binary_map(self, prices):
#         output = pd.DataFrame()
#         for i in prices.columns:
#             # binary classification of returns
            
#             up_down = pd.DataFrame(prices[i].pct_change().dropna())
#             up_down[i] = up_down[i].map(model_validation.binary_classificator)


#             # concatenate all companies in dataframe
#             output = pd.concat([output, up_down], axis=1)

#         return output
    
    
    
#     def train_test_split(self, window, val_size, test=False, test_size=False):
#         self.window = window
#         self.val_size = val_size

        
#         self.ohlc_val = self.ohlc[:self.window + self.val_size]
#         self.tech_val = self.tech[:self.window + self.val_size]
#         self.feat_val = self.features[:self.window + self.val_size]
#         self.binary_val = model_validation.binary_map(self.ohlc['Adj Close'])[:self.window + self.val_size - 1]
#         self.returns_val = 100 * self.ohlc['Adj Close'].pct_change().dropna()[:self.window + self.val_size - 1]

#         print('Initial training window is included')
#         print(f'ohlc val {len(self.ohlc_val)}, tech val {len(self.tech_val)}')
#         print(f'binary_val {len(self.binary_val)}, returns_val {len(self.returns_val)}')
#         assert len(self.ohlc_val) == len(self.tech_val)
#         assert len(self.binary_val) == len(self.returns_val)
#         assert len(self.ohlc_val) == len(self.binary_val) + 1
#         assert self.ohlc_val.index[1] == self.binary_val.index[0]
        
#         assert self.feat_val.index[0] == self.tech_val.index[0]

#         if test:
            
#             self.ohlc_test = self.ohlc[self.window + self.val_size:]
#             self.tech_test = self.tech[self.window + self.val_size:]
#             self.feat_test = self.features[self.window + self.val_size:]
#             self.binary_test = model_validation.binary_map(self.ohlc['Adj Close'])[self.window + self.val_size :]
#             self.returns_test = 100 * self.ohlc['Adj Close'].pct_change().dropna()[self.window + self.val_size :]
#             self.test_size = len(self.ohlc_test) - self.window
            
#             print('Initial training window is included')
#             print(f'ohlc test {len(self.ohlc_test)}, tech test {len(self.tech_test)}')
#             print(f'binary_test {len(self.binary_test)}, returns_test {len(self.returns_test)}')
            
#             assert len(self.ohlc_test) == len(self.tech_test)
#             assert len(self.binary_test) == len(self.returns_test)
#             assert len(self.ohlc_test) == len(self.binary_test) + 1
#             assert self.ohlc_test.index[1] == self.binary_test.index[0]
#             assert self.feat_test.index[0] == self.tech_test.index[0]
        
        
#     def classify_next_bar(self, X, y, RF_params=(10, None, None), features=None):
#         """
#         X, y - features and target-vector based on a shape of rolling window
#         X - tech features and external data for the date t-1
#         y - binary target - [1 - up, -1 - down] for time t
#         X_out - 1 day forward features to make prediction
#         Make assert statement to ensure that
        
#         """
#         assert X.shape[0] - 1 == y.shape[0]
#         assert len(X) == len(features)
#         X_data = pd.concat([X, features], axis=1)
#         X_train_data = X_data.shift().dropna()
        
#         assert X_train_data.shape[0] == y.shape[0]
        
#         X_forecast = pd.DataFrame(X_data.iloc[-1, :]).T
        
#         # Fit the model - returns classifier
#         model = RandomForestRegressor(n_estimators=RF_params[0], 
#                                        max_depth=RF_params[1], 
#                                        max_leaf_nodes=RF_params[2], 
#                                        random_state=42,
#                                        n_jobs=-1)
#         model.fit(X_train_data, y)
#         bar = model.predict(X_forecast)
        
#         return bar
    
#     def GARCH_model(self, returns, GARCH_params=(1, 1), summary=False):
#         assert type(returns) == pd.Series
#         model = arch_model(
#             returns,
#             p=GARCH_params[0],
#             q=GARCH_params[1],
#             rescale=False
#         )

#         model_res = model.fit(disp=0)
#         next_pred = model_res.forecast(horizon=1, reindex=False).variance.iloc[0,0] ** 0.5
        
#         if summary:
#             aic = model_res.aic
#             return next_pred, aic

#         return next_pred
    
    
    
#     def rolling_window(self, tech, binary, window, test_size, returns, 
#                        val_model='Both', RF_params=(10, None, None), 
#                        GARCH_params=(1, 1), summary_garch=False, features=None):
#         """
#         val_model: what model to validate. Relevant parameters: 'RF', 'GARCH', 'Both'
        
#         """
#         if val_model == 'Both':


#             prediction_ret = pd.DataFrame(columns=binary.columns)
#             prediction_std = pd.DataFrame(columns=returns.columns)

#             total_time = time.time()

#             for i in range(-test_size, 0):
#                 start_time = time.time()
#                 # Create rolling window to train the model
#                 # 100 - window size, -1 correction for lag of tech features
#                 X_train = tech.iloc[-window-1+i:i]
#                 y_train = binary.iloc[-window+i:i]

#                 y_train_ret = returns.iloc[-window+i:i]

#                 # RETURNS PART
#                 #list for returns predictions
#                 pred_list = np.array([])

#                 #VOLATILITY PART
#                 pred_list_std = np.array([])

#                 for company in binary.columns:
#                     # RETURNS PART
#                     # Train a model
#                     #print(f' Start {company}')

#                     next_pred = model_validation.classify_next_bar(X_train[company], y_train[company], RF_params)
#                     # Collect results and append them
#                     pred_list = np.append(pred_list, next_pred)


#                     #VOLATILITY PART
#                     # Train a model
#                     next_pred_std = model_validation.GARCH_model(y_train_ret[company], GARCH_params, summary=summary_garch)
#                     # Collect results and append them
#                     pred_list_std = np.append(pred_list_std, next_pred_std)

#                     #print(f' End {company}')

#                 pred_list = pd.DataFrame(pred_list, index=binary.columns).T
#                 prediction_ret = pd.concat([prediction_ret, pred_list], axis=0)

#                 pred_list_std = pd.DataFrame(pred_list_std, index=returns.columns).T
#                 prediction_std = pd.concat([prediction_std, pred_list_std], axis=0)
#                 elapsed_time = round(time.time() - start_time, 2)
#                 print(f"Test circle {i} trained in {elapsed_time} seconds.")



#             prediction_ret.index = binary.iloc[-test_size:, :].index
#             prediction_std.index = returns.iloc[-test_size:, :].index

#             elapsed_total_time = round(time.time() - total_time, 2)
#             print(f"Model trained in {elapsed_total_time} seconds.")

#             return prediction_ret, prediction_std
        
#         if val_model == 'RF':

#             prediction_ret = pd.DataFrame(columns=binary.columns)

#             total_time = time.time()

#             for i in range(-test_size, 0):
#                 start_time = time.time()
#                 # Create rolling window to train the model
#                 # 100 - window size, -1 correction for lag of tech features
#                 X_train = tech.iloc[-window-1+i:i]
#                 if features is None:
#                     pass
#                 else:
#                     X_features = features.iloc[-window-1+i:i]
#                     assert len(X_train) == len(X_features)
                
#                 y_train = returns.iloc[-window+i:i]

#                 # RETURNS PART
#                 #list for returns predictions
#                 pred_list = np.array([])

#                 for company in binary.columns:
#                     # RETURNS PART
#                     # Train a model
#                     if features is None:
#                         next_pred = model_validation.classify_next_bar(X_train[company], y_train[company], 
#                                                                    RF_params)
#                     else:
#                         next_pred = model_validation.classify_next_bar(X_train[company], y_train[company], 
#                                                                    RF_params, features=X_features)
                    
#                     # Collect results and append them
#                     pred_list = np.append(pred_list, next_pred)

#                 pred_list = pd.DataFrame(pred_list, index=binary.columns).T
#                 prediction_ret = pd.concat([prediction_ret, pred_list], axis=0)

#                 elapsed_time = round(time.time() - start_time, 2)
#                 print(f"Test circle {i} trained in {elapsed_time} seconds.")

#             prediction_ret.index = binary.iloc[-test_size:, :].index

#             elapsed_total_time = round(time.time() - total_time, 2)
#             print(f"Model trained in {elapsed_total_time} seconds.")

#             return prediction_ret
        
        
#         if val_model == 'GARCH':
#             if summary_garch:
#                 aic = {}
            
#             prediction_std = pd.DataFrame(columns=returns.columns)

#             total_time = time.time()

#             for i in range(-test_size, 0):
#                 start_time = time.time()
#                 # Create rolling window to train the model

#                 y_train_ret = returns.iloc[-window+i:i]

#                 #VOLATILITY PART
#                 pred_list_std = np.array([])

#                 for company in binary.columns:

#                     #VOLATILITY PART
#                     # Train a model
#                     if summary_garch:
#                         next_pred_std, aic[(i, company)] = model_validation.GARCH_model(y_train_ret[company], 
#                                                                      GARCH_params, 
#                                                                      summary=True)
#                     else:
#                         next_pred_std = model_validation.GARCH_model(y_train_ret[company], 
#                                                                      GARCH_params, 
#                                                                      summary=False)
                    
                    
                    
#                     # Collect results and append them
#                     pred_list_std = np.append(pred_list_std, next_pred_std)
                    
#                 pred_list_std = pd.DataFrame(pred_list_std, index=returns.columns).T
#                 prediction_std = pd.concat([prediction_std, pred_list_std], axis=0)
#                 elapsed_time = round(time.time() - start_time, 2)
#                 print(f"Test circle {i} trained in {elapsed_time} seconds.")

#             prediction_std.index = returns.iloc[-test_size:, :].index

#             elapsed_total_time = round(time.time() - total_time, 2)
#             print(f"Model trained in {elapsed_total_time} seconds.")
            
#             if summary_garch:
#                 return prediction_std, aic
            
#             return prediction_std            
              
        
        
#     def get_return_predictions(self, tech, binary, window, test_size, returns):
#         direction, standard_dev = model_validation.rolling_window(tech, binary, window, test_size, returns)
#         returns_prediction = direction * standard_dev

#         return returns_prediction / 100, direction
        
        
    
    

    
    

In [3]:
class model_validation:
    
    def __init__(self, tickers, start, end, features_tickers):
        self.tickers = tickers
        self.start = start
        self.end = end

        self.features = yf.download(features_tickers, start=self.start, end=self.end, 
                                    interval='1wk').fillna(method='ffill')
        self.ohlc = yf.download(self.tickers, start=self.start, end=self.end, interval='1wk')
    
   
    def tech_features(self):
        inst = dw()
        output = pd.DataFrame()
        for i in self.ohlc.columns.levels[1]:

            tech = inst.get_tech_features(close=self.ohlc['Adj Close'][i],
                                              volume=self.ohlc['Volume'][i],
                                              low=self.ohlc['Low'][i],
                                              high=self.ohlc['High'][i],
                                              concat=True).dropna()
            tech.columns = pd.MultiIndex.from_product([[i], tech.columns])

            output = pd.concat([output, tech], axis=1)
            
        self.ohlc = self.ohlc.loc[output.index]
        self.features = self.features['Adj Close'].loc[output.index]
        self.tech = output
        print('Done!')
        

    def binary_classificator(self, x):
        if x > 0:
            return 1
        else:
            return -1
        
    
    def binary_map(self, prices):
        output = pd.DataFrame()
        for i in prices.columns:
            # binary classification of returns
            
            up_down = pd.DataFrame(prices[i].pct_change().dropna())
            up_down[i] = up_down[i].map(self.binary_classificator)


            # concatenate all companies in dataframe
            output = pd.concat([output, up_down], axis=1)

        return output
    
    
    
    def train_test_split(self, window, val_size, test=False, test_size=False):
        self.window = window
        self.val_size = val_size

        
        self.ohlc_val = self.ohlc[:self.window + self.val_size]
        self.tech_val = self.tech[:self.window + self.val_size]
        self.feat_val = self.features[:self.window + self.val_size]
        self.binary_val = self.binary_map(prices=self.ohlc['Adj Close'])[:self.window + self.val_size - 1]
        self.returns_val = 100 * self.ohlc['Adj Close'].pct_change().dropna()[:self.window + self.val_size - 1]

        print('Initial training window is included')
        print(f'ohlc val {len(self.ohlc_val)}, tech val {len(self.tech_val)}')
        print(f'binary_val {len(self.binary_val)}, returns_val {len(self.returns_val)}')
        assert len(self.ohlc_val) == len(self.tech_val)
        assert len(self.binary_val) == len(self.returns_val)
        assert len(self.ohlc_val) == len(self.binary_val) + 1
        assert self.ohlc_val.index[1] == self.binary_val.index[0]
        
        assert self.feat_val.index[0] == self.tech_val.index[0]

        if test:
            
            self.ohlc_test = self.ohlc[self.window + self.val_size:]
            self.tech_test = self.tech[self.window + self.val_size:]
            self.feat_test = self.features[self.window + self.val_size:]
            self.binary_test = self.binary_map(self.ohlc['Adj Close'])[self.window + self.val_size :]
            self.returns_test = 100 * self.ohlc['Adj Close'].pct_change().dropna()[self.window + self.val_size :]
            self.test_size = len(self.ohlc_test) - self.window
            
            print('Initial training window is included')
            print(f'ohlc test {len(self.ohlc_test)}, tech test {len(self.tech_test)}')
            print(f'binary_test {len(self.binary_test)}, returns_test {len(self.returns_test)}')
            
            assert len(self.ohlc_test) == len(self.tech_test)
            assert len(self.binary_test) == len(self.returns_test)
            assert len(self.ohlc_test) == len(self.binary_test) + 1
            assert self.ohlc_test.index[1] == self.binary_test.index[0]
            assert self.feat_test.index[0] == self.tech_test.index[0]
        
        
    def classify_next_bar(self, X, y, RF_params=(10, None, None), features=None):
        """
        X, y - features and target-vector based on a shape of rolling window
        X - tech features and external data for the date t-1
        y - binary target - [1 - up, -1 - down] for time t
        X_out - 1 day forward features to make prediction
        Make assert statement to ensure that
        
        """
        assert X.shape[0] - 1 == y.shape[0]
        assert len(X) == len(features)
        X_data = pd.concat([X, features], axis=1)
        X_train_data = X_data.shift().dropna()
        
        assert X_train_data.shape[0] == y.shape[0]
        
        X_forecast = pd.DataFrame(X_data.iloc[-1, :]).T
        
        # Fit the model - returns classifier
        model = RandomForestClassifier(n_estimators=RF_params[0], 
                                       max_depth=RF_params[1], 
                                       max_leaf_nodes=RF_params[2], 
                                       random_state=42,
                                       n_jobs=-1)
        model.fit(X_train_data, y)
        bar = model.predict(X_forecast)
        
        return bar

    def regress_next_bar(self, X, y, RF_params=(10, None, None), features=None):
        """
        X, y - features and target-vector based on a shape of rolling window
        X - tech features and external data for the date t-1
        y - binary target - [1 - up, -1 - down] for time t
        X_out - 1 day forward features to make prediction
        Make assert statement to ensure that
        
        """
        assert X.shape[0] - 1 == y.shape[0]
        assert len(X) == len(features)
        X_data = pd.concat([X, features], axis=1)
        X_train_data = X_data.shift().dropna()
        
        assert X_train_data.shape[0] == y.shape[0]
        
        X_forecast = pd.DataFrame(X_data.iloc[-1, :]).T
        
        # Fit the model - returns classifier
        model = RandomForestRegressor(n_estimators=RF_params[0], 
                                       max_depth=RF_params[1], 
                                       max_leaf_nodes=RF_params[2], 
                                       random_state=42,
                                       n_jobs=-1)
        model.fit(X_train_data, y)
        bar = model.predict(X_forecast)
        
        return bar    
    
    
    
    
    
    def GARCH_model(self, returns, GARCH_params=(1, 1), summary=False):
        assert type(returns) == pd.Series
        model = arch_model(
            returns,
            p=GARCH_params[0],
            q=GARCH_params[1],
            rescale=False
        )

        model_res = model.fit(disp=0)
        next_pred = model_res.forecast(horizon=1, reindex=False).variance.iloc[0,0] ** 0.5
        
        if summary:
            aic = model_res.aic
            return next_pred, aic

        return next_pred
    
    
    def expanding_window(self, tech, binary, window, test_size, returns,
                        val_model='RF', RF_params=(10, None, None),
                        GARCH_params=(1, 1), summary_garch=False, features=None):
        
        if val_model == 'RF':

            prediction_ret = pd.DataFrame(columns=binary.columns)

            total_time = time.time()

            for i in range(-test_size, 0):
                start_time = time.time()
                # Create rolling window to train the model
                # 100 - window size, -1 correction for lag of tech features
                X_train = tech.iloc[-window-1+i:i]
                if features is None:
                    pass
                else:
                    X_features = features.iloc[-window-1+i:i]
                    assert len(X_train) == len(X_features)
                
                y_train = returns.iloc[-window+i:i]

                # RETURNS PART
                #list for returns predictions
                pred_list = np.array([])

                for company in binary.columns:
                    # RETURNS PART
                    # Train a model
                    if features is None:
                        next_pred = self.regress_next_bar(X_train[company], y_train[company], 
                                                                   RF_params)
                    else:
                        next_pred = self.regress_next_bar(X_train[company], y_train[company], 
                                                                   RF_params, features=X_features)
                    
                    # Collect results and append them
                    pred_list = np.append(pred_list, next_pred)

                pred_list = pd.DataFrame(pred_list, index=binary.columns).T
                prediction_ret = pd.concat([prediction_ret, pred_list], axis=0)

                elapsed_time = round(time.time() - start_time, 2)
                print(f"Test circle {i} trained in {elapsed_time} seconds.")

            prediction_ret.index = binary.iloc[-test_size:, :].index

            elapsed_total_time = round(time.time() - total_time, 2)
            print(f"Model trained in {elapsed_total_time} seconds.")

            return prediction_ret        
    
    
    
    def rolling_window(self, tech, binary, window, test_size, returns, 
                       val_model='Both', RF_params=(10, None, None), 
                       GARCH_params=(1, 1), summary_garch=False, features=None):
        """
        val_model: what model to validate. Relevant parameters: 'RF', 'GARCH', 'Both'
        
        """
        if val_model == 'Both':


            prediction_ret = pd.DataFrame(columns=binary.columns)
            prediction_std = pd.DataFrame(columns=returns.columns)

            total_time = time.time()

            for i in range(-test_size, 0):
                start_time = time.time()
                # Create rolling window to train the model
                # 100 - window size, -1 correction for lag of tech features
                X_train = tech.iloc[-window-1+i:i]
                y_train = binary.iloc[-window+i:i]

                y_train_ret = returns.iloc[-window+i:i]

                # RETURNS PART
                #list for returns predictions
                pred_list = np.array([])

                #VOLATILITY PART
                pred_list_std = np.array([])

                for company in binary.columns:
                    # RETURNS PART
                    # Train a model
                    #print(f' Start {company}')

                    next_pred = self.classify_next_bar(X_train[company], y_train[company], RF_params)
                    # Collect results and append them
                    pred_list = np.append(pred_list, next_pred)


                    #VOLATILITY PART
                    # Train a model
                    next_pred_std = self.GARCH_model(y_train_ret[company], GARCH_params, summary=summary_garch)
                    # Collect results and append them
                    pred_list_std = np.append(pred_list_std, next_pred_std)

                    #print(f' End {company}')

                pred_list = pd.DataFrame(pred_list, index=binary.columns).T
                prediction_ret = pd.concat([prediction_ret, pred_list], axis=0)

                pred_list_std = pd.DataFrame(pred_list_std, index=returns.columns).T
                prediction_std = pd.concat([prediction_std, pred_list_std], axis=0)
                elapsed_time = round(time.time() - start_time, 2)
                print(f"Test circle {i} trained in {elapsed_time} seconds.")



            prediction_ret.index = binary.iloc[-test_size:, :].index
            prediction_std.index = returns.iloc[-test_size:, :].index

            elapsed_total_time = round(time.time() - total_time, 2)
            print(f"Model trained in {elapsed_total_time} seconds.")

            return prediction_ret, prediction_std
        
        if val_model == 'RF':

            prediction_ret = pd.DataFrame(columns=binary.columns)

            total_time = time.time()

            for i in range(-test_size, 0):
                start_time = time.time()
                # Create rolling window to train the model
                # 100 - window size, -1 correction for lag of tech features
                X_train = tech.iloc[-window-1+i:i]
                if features is None:
                    pass
                else:
                    X_features = features.iloc[-window-1+i:i]
                    assert len(X_train) == len(X_features)
                
                y_train = binary.iloc[-window+i:i]

                # RETURNS PART
                #list for returns predictions
                pred_list = np.array([])

                for company in binary.columns:
                    # RETURNS PART
                    # Train a model
                    if features is None:
                        next_pred = self.classify_next_bar(X_train[company], y_train[company], 
                                                                   RF_params)
                    else:
                        next_pred = self.classify_next_bar(X_train[company], y_train[company], 
                                                                   RF_params, features=X_features)
                    
                    # Collect results and append them
                    pred_list = np.append(pred_list, next_pred)

                pred_list = pd.DataFrame(pred_list, index=binary.columns).T
                prediction_ret = pd.concat([prediction_ret, pred_list], axis=0)

                elapsed_time = round(time.time() - start_time, 2)
                print(f"Test circle {i} trained in {elapsed_time} seconds.")

            prediction_ret.index = binary.iloc[-test_size:, :].index

            elapsed_total_time = round(time.time() - total_time, 2)
            print(f"Model trained in {elapsed_total_time} seconds.")

            return prediction_ret
        
        
        if val_model == 'GARCH':
            if summary_garch:
                aic = {}
            
            prediction_std = pd.DataFrame(columns=returns.columns)

            total_time = time.time()

            for i in range(-test_size, 0):
                start_time = time.time()
                # Create rolling window to train the model

                y_train_ret = returns.iloc[-window+i:i]

                #VOLATILITY PART
                pred_list_std = np.array([])

                for company in binary.columns:

                    #VOLATILITY PART
                    # Train a model
                    if summary_garch:
                        next_pred_std, aic[(i, company)] = self.GARCH_model(y_train_ret[company], 
                                                                     GARCH_params, 
                                                                     summary=True)
                    else:
                        next_pred_std = self.GARCH_model(y_train_ret[company], 
                                                                     GARCH_params, 
                                                                     summary=False)
                    
                    
                    
                    # Collect results and append them
                    pred_list_std = np.append(pred_list_std, next_pred_std)
                    
                pred_list_std = pd.DataFrame(pred_list_std, index=returns.columns).T
                prediction_std = pd.concat([prediction_std, pred_list_std], axis=0)
                elapsed_time = round(time.time() - start_time, 2)
                print(f"Test circle {i} trained in {elapsed_time} seconds.")

            prediction_std.index = returns.iloc[-test_size:, :].index

            elapsed_total_time = round(time.time() - total_time, 2)
            print(f"Model trained in {elapsed_total_time} seconds.")
            
            if summary_garch:
                return prediction_std, aic
            
            return prediction_std            
              
        
        
    def get_return_predictions(self, tech, binary, window, test_size, returns):
        direction, standard_dev = self.rolling_window(tech, binary, window, test_size, returns)
        returns_prediction = direction * standard_dev

        return returns_prediction / 100, direction
        
        
    
    

    
    

In [5]:
# Validation
tickers = ['AMZN', 'TM', 'HD', 'GOOGL', 'VZ', 'CMCSA',
          'WMT', 'PG', 'KO', 'XOM', 'CVX', 'SHEL',
          'SPGI', 'WFC', 'JPM', 'UPS', 'RTX', 'HON',
          'UNH', 'LLY', 'JNJ', 'BHP', 'LIN', 'RIO',
          'PLD', 'AMT', 'EQIX', 'TSM', 'ORCL', 'CSCO',
          'NEE', 'SO', 'DUK']


# Test set
# tickers = ['MCD', 'NKE', 'SBUX', 'DIS', 'T', 'NFLX',
#           'PEP', 'COST', 'UL', 'TTE', 'BP', 'EQNR',
#           'BAC', 'HSBC', 'MS', 'BA', 'LMT', 'GE',
#           'AZN', 'MRK', 'PFE', 'APD', 'VALE', 'SHW',
#           'PSA', 'CCI', 'O', 'AAPL', 'MSFT', 'NVDA',
#           'PCG', 'SRE', 'AEP']

In [6]:


features = ["EURUSD=X", '^IRX', '^FVX', 'CL=F', 'GC=F', 'NG=F', 'RB=F']
start = '2005-01-01'
end = '2022-12-31'

In [7]:
model_validation = model_validation(tickers, start, end, features)

[*********************100%***********************]  7 of 7 completed
[*********************100%***********************]  33 of 33 completed


In [10]:
print(len(model_validation.ohlc))

939


In [11]:
model_validation.tech_features()

Done!


In [13]:
window = int(len(model_validation.tech) * 0.2)
val_size = len(model_validation.tech) - window
print(window)
print(val_size)

181
725


In [14]:
model_validation.train_test_split(window=window, val_size=val_size, test=False)

Initial training window is included
ohlc val 906, tech val 906
binary_val 905, returns_val 905


# Random Forest Validation

In [17]:
# Random forest params
RF_params = []
param_grid_random_forest = {
    'n_estimators': [25, 50],
    'max_depth': [3, 6],
    'max_leaf_nodes': [3, 6],
}
# param_grid_random_forest = {
#     'n_estimators': [50],
#     'max_depth': [3],
#     'max_leaf_nodes': [None]
# }

for i in itertools.product(param_grid_random_forest['n_estimators'],
                           param_grid_random_forest['max_depth'],
                           param_grid_random_forest['max_leaf_nodes']):
    
    RF_params.append(i)

In [18]:
random_forest_val_results = []
time_to_train_RF = {}

for i in RF_params:
    total_time = time.time()
    
    print('=========================================================================================')
    print(f'Model {i} starts')
    print('=========================================================================================')
    direction = model_validation.rolling_window(model_validation.tech_val, 
                                            model_validation.binary_val, 
                                            model_validation.window, 
                                            model_validation.val_size, 
                                            model_validation.returns_val,
                                            val_model='RF',
                                            RF_params=i,
                                            features=model_validation.feat_val)
    
    random_forest_val_results.append(direction)
    
    elapsed_time = round(time.time() - total_time, 2)
    print(f"Hyperparams {i} trained in {elapsed_time} seconds.")    
    time_to_train_RF[i] = f'{elapsed_time} seconds'
    print('=========================================================================================')
    print(f'Model {i} Ended')
    print('=========================================================================================')
    
    

Model (25, 3, 3) starts
Test circle -725 trained in 1.51 seconds.
Test circle -724 trained in 1.54 seconds.
Test circle -723 trained in 1.55 seconds.
Test circle -722 trained in 1.53 seconds.
Test circle -721 trained in 1.59 seconds.
Test circle -720 trained in 1.54 seconds.
Test circle -719 trained in 1.57 seconds.
Test circle -718 trained in 1.65 seconds.
Test circle -717 trained in 1.57 seconds.
Test circle -716 trained in 1.54 seconds.
Test circle -715 trained in 1.57 seconds.
Test circle -714 trained in 1.73 seconds.
Test circle -713 trained in 1.64 seconds.
Test circle -712 trained in 1.63 seconds.
Test circle -711 trained in 1.63 seconds.
Test circle -710 trained in 1.63 seconds.
Test circle -709 trained in 1.54 seconds.
Test circle -708 trained in 1.59 seconds.
Test circle -707 trained in 1.59 seconds.
Test circle -706 trained in 1.54 seconds.
Test circle -705 trained in 1.55 seconds.
Test circle -704 trained in 1.56 seconds.
Test circle -703 trained in 1.61 seconds.
Test circl

Test circle -534 trained in 1.67 seconds.
Test circle -533 trained in 1.67 seconds.
Test circle -532 trained in 1.56 seconds.
Test circle -531 trained in 1.55 seconds.
Test circle -530 trained in 1.58 seconds.
Test circle -529 trained in 1.6 seconds.
Test circle -528 trained in 1.62 seconds.
Test circle -527 trained in 1.56 seconds.
Test circle -526 trained in 1.58 seconds.
Test circle -525 trained in 1.58 seconds.
Test circle -524 trained in 1.64 seconds.
Test circle -523 trained in 1.59 seconds.
Test circle -522 trained in 1.56 seconds.
Test circle -521 trained in 1.58 seconds.
Test circle -520 trained in 1.56 seconds.
Test circle -519 trained in 1.61 seconds.
Test circle -518 trained in 1.61 seconds.
Test circle -517 trained in 1.56 seconds.
Test circle -516 trained in 1.57 seconds.
Test circle -515 trained in 1.61 seconds.
Test circle -514 trained in 1.73 seconds.
Test circle -513 trained in 1.58 seconds.
Test circle -512 trained in 1.57 seconds.
Test circle -511 trained in 1.56 se

Test circle -338 trained in 1.69 seconds.
Test circle -337 trained in 1.72 seconds.
Test circle -336 trained in 1.81 seconds.
Test circle -335 trained in 1.66 seconds.
Test circle -334 trained in 1.67 seconds.
Test circle -333 trained in 1.7 seconds.
Test circle -332 trained in 1.74 seconds.
Test circle -331 trained in 1.76 seconds.
Test circle -330 trained in 1.94 seconds.
Test circle -329 trained in 1.78 seconds.
Test circle -328 trained in 1.79 seconds.
Test circle -327 trained in 1.67 seconds.
Test circle -326 trained in 1.66 seconds.
Test circle -325 trained in 1.68 seconds.
Test circle -324 trained in 1.69 seconds.
Test circle -323 trained in 1.65 seconds.
Test circle -322 trained in 1.63 seconds.
Test circle -321 trained in 1.63 seconds.
Test circle -320 trained in 1.73 seconds.
Test circle -319 trained in 1.72 seconds.
Test circle -318 trained in 1.63 seconds.
Test circle -317 trained in 1.61 seconds.
Test circle -316 trained in 1.76 seconds.
Test circle -315 trained in 1.73 se

Test circle -142 trained in 1.61 seconds.
Test circle -141 trained in 1.61 seconds.
Test circle -140 trained in 1.61 seconds.
Test circle -139 trained in 1.61 seconds.
Test circle -138 trained in 1.61 seconds.
Test circle -137 trained in 1.82 seconds.
Test circle -136 trained in 1.91 seconds.
Test circle -135 trained in 1.71 seconds.
Test circle -134 trained in 1.69 seconds.
Test circle -133 trained in 1.62 seconds.
Test circle -132 trained in 1.67 seconds.
Test circle -131 trained in 1.62 seconds.
Test circle -130 trained in 1.63 seconds.
Test circle -129 trained in 1.65 seconds.
Test circle -128 trained in 1.63 seconds.
Test circle -127 trained in 1.63 seconds.
Test circle -126 trained in 1.63 seconds.
Test circle -125 trained in 1.61 seconds.
Test circle -124 trained in 1.62 seconds.
Test circle -123 trained in 1.65 seconds.
Test circle -122 trained in 1.61 seconds.
Test circle -121 trained in 1.61 seconds.
Test circle -120 trained in 1.6 seconds.
Test circle -119 trained in 1.63 se

Test circle -680 trained in 1.69 seconds.
Test circle -679 trained in 1.72 seconds.
Test circle -678 trained in 1.69 seconds.
Test circle -677 trained in 1.65 seconds.
Test circle -676 trained in 1.64 seconds.
Test circle -675 trained in 1.63 seconds.
Test circle -674 trained in 1.76 seconds.
Test circle -673 trained in 1.68 seconds.
Test circle -672 trained in 1.66 seconds.
Test circle -671 trained in 1.73 seconds.
Test circle -670 trained in 1.82 seconds.
Test circle -669 trained in 1.66 seconds.
Test circle -668 trained in 1.65 seconds.
Test circle -667 trained in 1.65 seconds.
Test circle -666 trained in 1.61 seconds.
Test circle -665 trained in 1.69 seconds.
Test circle -664 trained in 1.63 seconds.
Test circle -663 trained in 1.62 seconds.
Test circle -662 trained in 1.67 seconds.
Test circle -661 trained in 1.64 seconds.
Test circle -660 trained in 1.65 seconds.
Test circle -659 trained in 1.73 seconds.
Test circle -658 trained in 2.04 seconds.
Test circle -657 trained in 1.83 s

...

Test circle -230 trained in 2.76 seconds.
Test circle -229 trained in 2.74 seconds.
Test circle -228 trained in 2.77 seconds.
Test circle -227 trained in 2.75 seconds.
Test circle -226 trained in 2.74 seconds.
Test circle -225 trained in 2.74 seconds.
Test circle -224 trained in 2.75 seconds.
Test circle -223 trained in 2.74 seconds.
Test circle -222 trained in 2.78 seconds.
Test circle -221 trained in 3.18 seconds.
Test circle -220 trained in 3.24 seconds.
Test circle -219 trained in 3.66 seconds.
Test circle -218 trained in 3.46 seconds.
Test circle -217 trained in 2.99 seconds.
Test circle -216 trained in 2.88 seconds.
Test circle -215 trained in 3.25 seconds.
Test circle -214 trained in 3.98 seconds.
Test circle -213 trained in 3.1 seconds.
Test circle -212 trained in 2.85 seconds.
Test circle -211 trained in 2.83 seconds.
Test circle -210 trained in 2.91 seconds.
Test circle -209 trained in 2.82 seconds.
Test circle -208 trained in 2.76 seconds.
Test circle -207 trained in 2.76 se

Test circle -33 trained in 3.19 seconds.
Test circle -32 trained in 3.06 seconds.
Test circle -31 trained in 2.98 seconds.
Test circle -30 trained in 2.8 seconds.
Test circle -29 trained in 2.81 seconds.
Test circle -28 trained in 2.77 seconds.
Test circle -27 trained in 2.84 seconds.
Test circle -26 trained in 2.77 seconds.
Test circle -25 trained in 2.74 seconds.
Test circle -24 trained in 2.75 seconds.
Test circle -23 trained in 2.79 seconds.
Test circle -22 trained in 2.74 seconds.
Test circle -21 trained in 2.73 seconds.
Test circle -20 trained in 2.75 seconds.
Test circle -19 trained in 2.73 seconds.
Test circle -18 trained in 2.73 seconds.
Test circle -17 trained in 2.97 seconds.
Test circle -16 trained in 2.84 seconds.
Test circle -15 trained in 2.89 seconds.
Test circle -14 trained in 2.84 seconds.
Test circle -13 trained in 2.9 seconds.
Test circle -12 trained in 2.87 seconds.
Test circle -11 trained in 3.59 seconds.
Test circle -10 trained in 3.8 seconds.
Test circle -9 trai

Test circle -573 trained in 3.56 seconds.
Test circle -572 trained in 3.57 seconds.
Test circle -571 trained in 3.59 seconds.
Test circle -570 trained in 3.6 seconds.
Test circle -569 trained in 3.58 seconds.
Test circle -568 trained in 3.59 seconds.
Test circle -567 trained in 3.67 seconds.
Test circle -566 trained in 3.59 seconds.
Test circle -565 trained in 3.92 seconds.
Test circle -564 trained in 3.58 seconds.
Test circle -563 trained in 3.54 seconds.
Test circle -562 trained in 3.52 seconds.
Test circle -561 trained in 3.59 seconds.
Test circle -560 trained in 3.59 seconds.
Test circle -559 trained in 3.57 seconds.
Test circle -558 trained in 3.57 seconds.
Test circle -557 trained in 3.58 seconds.
Test circle -556 trained in 3.53 seconds.
Test circle -555 trained in 3.57 seconds.
Test circle -554 trained in 3.54 seconds.
Test circle -553 trained in 3.61 seconds.
Test circle -552 trained in 3.64 seconds.
Test circle -551 trained in 3.71 seconds.
Test circle -550 trained in 3.6 sec

Test circle -377 trained in 2.72 seconds.
Test circle -376 trained in 2.77 seconds.
Test circle -375 trained in 2.79 seconds.
Test circle -374 trained in 2.73 seconds.
Test circle -373 trained in 2.75 seconds.
Test circle -372 trained in 2.76 seconds.
Test circle -371 trained in 2.71 seconds.
Test circle -370 trained in 2.75 seconds.
Test circle -369 trained in 2.8 seconds.
Test circle -368 trained in 2.73 seconds.
Test circle -367 trained in 2.75 seconds.
Test circle -366 trained in 2.85 seconds.
Test circle -365 trained in 2.84 seconds.
Test circle -364 trained in 2.82 seconds.
Test circle -363 trained in 2.91 seconds.
Test circle -362 trained in 2.81 seconds.
Test circle -361 trained in 2.88 seconds.
Test circle -360 trained in 2.89 seconds.
Test circle -359 trained in 2.85 seconds.
Test circle -358 trained in 2.92 seconds.
Test circle -357 trained in 2.92 seconds.
Test circle -356 trained in 2.91 seconds.
Test circle -355 trained in 2.97 seconds.
Test circle -354 trained in 2.99 se

Test circle -180 trained in 2.85 seconds.
Test circle -179 trained in 2.88 seconds.
Test circle -178 trained in 2.79 seconds.
Test circle -177 trained in 2.81 seconds.
Test circle -176 trained in 2.79 seconds.
Test circle -175 trained in 2.79 seconds.
Test circle -174 trained in 2.79 seconds.
Test circle -173 trained in 2.79 seconds.
Test circle -172 trained in 2.78 seconds.
Test circle -171 trained in 2.78 seconds.
Test circle -170 trained in 2.83 seconds.
Test circle -169 trained in 2.79 seconds.
Test circle -168 trained in 2.79 seconds.
Test circle -167 trained in 2.79 seconds.
Test circle -166 trained in 2.78 seconds.
Test circle -165 trained in 2.79 seconds.
Test circle -164 trained in 2.78 seconds.
Test circle -163 trained in 2.79 seconds.
Test circle -162 trained in 2.77 seconds.
Test circle -161 trained in 2.86 seconds.
Test circle -160 trained in 2.78 seconds.
Test circle -159 trained in 2.79 seconds.
Test circle -158 trained in 2.77 seconds.
Test circle -157 trained in 2.79 s

# GARCH Validation

In [22]:
# Random forest params
GARCH_params = []

# param_grid_garch = {
#     'p': [1, 2, 3, 4],
#     'q': [1, 2, 3, 4]
# }
param_grid_garch = {
    'p': [1],
    'q': [1]
}
for i in itertools.product(param_grid_garch['p'],
                           param_grid_garch['q']):
    
    GARCH_params.append(i)

In [23]:
GARCH_val_results = []
time_to_train_GARCH = {}
all_aic = []

for i in GARCH_params:
    total_time = time.time()
    
    print('=========================================================================================')
    print(f'Model {i} starts')
    print('=========================================================================================')
    standard_dev, aic = model_validation.rolling_window(model_validation.tech_val, 
                                                model_validation.binary_val, 
                                                model_validation.window, 
                                                model_validation.val_size, 
                                                model_validation.returns_val,
                                                val_model='GARCH',
                                                GARCH_params=i,
                                                summary_garch=True)
    
    all_aic.append(aic)
    GARCH_val_results.append(standard_dev)
    
    elapsed_time = round(time.time() - total_time, 2)
    print(f"Hyperparams {i} trained in {elapsed_time} seconds.")    
    time_to_train_GARCH[i] = f'{elapsed_time} seconds'
    print('=========================================================================================')
    print(f'Model {i} Ended')
    print('=========================================================================================')
    
   

Model (1, 1) starts
Test circle -725 trained in 0.49 seconds.
Test circle -724 trained in 0.47 seconds.
Test circle -723 trained in 0.47 seconds.
Test circle -722 trained in 0.46 seconds.
Test circle -721 trained in 0.48 seconds.
Test circle -720 trained in 0.47 seconds.
Test circle -719 trained in 0.47 seconds.
Test circle -718 trained in 0.48 seconds.
Test circle -717 trained in 0.47 seconds.
Test circle -716 trained in 0.47 seconds.
Test circle -715 trained in 0.46 seconds.
Test circle -714 trained in 0.45 seconds.
Test circle -713 trained in 0.47 seconds.
Test circle -712 trained in 0.47 seconds.
Test circle -711 trained in 0.47 seconds.
Test circle -710 trained in 0.48 seconds.
Test circle -709 trained in 0.44 seconds.
Test circle -708 trained in 0.45 seconds.
Test circle -707 trained in 0.45 seconds.
Test circle -706 trained in 0.46 seconds.
Test circle -705 trained in 0.47 seconds.
Test circle -704 trained in 0.44 seconds.
Test circle -703 trained in 0.47 seconds.
Test circle -7

Test circle -534 trained in 0.48 seconds.
Test circle -533 trained in 0.52 seconds.
Test circle -532 trained in 0.52 seconds.
Test circle -531 trained in 0.5 seconds.
Test circle -530 trained in 0.47 seconds.
Test circle -529 trained in 0.48 seconds.
Test circle -528 trained in 0.55 seconds.
Test circle -527 trained in 0.59 seconds.
Test circle -526 trained in 0.56 seconds.
Test circle -525 trained in 0.44 seconds.
Test circle -524 trained in 0.46 seconds.
Test circle -523 trained in 0.48 seconds.
Test circle -522 trained in 0.49 seconds.
Test circle -521 trained in 0.48 seconds.
Test circle -520 trained in 0.48 seconds.
Test circle -519 trained in 0.5 seconds.
Test circle -518 trained in 0.47 seconds.
Test circle -517 trained in 0.45 seconds.
Test circle -516 trained in 0.5 seconds.
Test circle -515 trained in 0.48 seconds.
Test circle -514 trained in 0.48 seconds.
Test circle -513 trained in 0.48 seconds.
Test circle -512 trained in 0.47 seconds.
Test circle -511 trained in 0.46 seco

Test circle -338 trained in 0.52 seconds.
Test circle -337 trained in 0.48 seconds.
Test circle -336 trained in 0.5 seconds.
Test circle -335 trained in 0.47 seconds.
Test circle -334 trained in 0.5 seconds.
Test circle -333 trained in 0.49 seconds.
Test circle -332 trained in 0.52 seconds.
Test circle -331 trained in 0.49 seconds.
Test circle -330 trained in 0.51 seconds.
Test circle -329 trained in 0.48 seconds.
Test circle -328 trained in 0.48 seconds.
Test circle -327 trained in 0.52 seconds.
Test circle -326 trained in 0.53 seconds.
Test circle -325 trained in 0.52 seconds.
Test circle -324 trained in 0.5 seconds.
Test circle -323 trained in 0.48 seconds.
Test circle -322 trained in 0.48 seconds.
Test circle -321 trained in 0.53 seconds.
Test circle -320 trained in 0.5 seconds.
Test circle -319 trained in 0.5 seconds.
Test circle -318 trained in 0.5 seconds.
Test circle -317 trained in 0.53 seconds.
Test circle -316 trained in 0.49 seconds.
Test circle -315 trained in 0.5 seconds.

Test circle -142 trained in 0.42 seconds.
Test circle -141 trained in 0.44 seconds.
Test circle -140 trained in 0.43 seconds.
Test circle -139 trained in 0.47 seconds.
Test circle -138 trained in 0.48 seconds.
Test circle -137 trained in 0.45 seconds.
Test circle -136 trained in 0.45 seconds.
Test circle -135 trained in 0.44 seconds.
Test circle -134 trained in 0.45 seconds.
Test circle -133 trained in 0.44 seconds.
Test circle -132 trained in 0.44 seconds.
Test circle -131 trained in 0.46 seconds.
Test circle -130 trained in 0.47 seconds.
Test circle -129 trained in 0.49 seconds.
Test circle -128 trained in 0.47 seconds.
Test circle -127 trained in 0.45 seconds.
Test circle -126 trained in 0.46 seconds.
Test circle -125 trained in 0.51 seconds.
Test circle -124 trained in 0.47 seconds.
Test circle -123 trained in 0.53 seconds.
Test circle -122 trained in 0.51 seconds.
Test circle -121 trained in 0.46 seconds.
Test circle -120 trained in 0.46 seconds.
Test circle -119 trained in 0.43 s

In [ ]:
def GARCH_model(self, returns, GARCH_params=(1, 1), summary=False):
    assert type(returns) == pd.Series
    model = arch_model(
        returns,
        p=GARCH_params[0],
        q=GARCH_params[1],
        rescale=False
    )

    model_res = model.fit(disp=0)
    next_pred = model_res.forecast(horizon=1, reindex=False).variance.iloc[0,0] ** 0.5

    if summary:
        aic = model_res.aic
        return next_pred, aic

    return next_pred

In [19]:

# for i in range(0, len(all_aic)):
#     print(f' Model {GARCH_params[i]} AIC Score {pd.Series(all_aic[i].values()).mean()}')


In [20]:
# Save validation results Random Forest


# pickle.dump(random_forest_val_results, open("random_forest_val_results.pkl", "wb"))
# pickle.dump(time_to_train_RF, open("time_to_train_RF.pkl", "wb"))
# pickle.dump(RF_params, open("RF_params.pkl", "wb"))
# pickle.dump(param_grid_random_forest, open("param_grid_random_forest.pkl", "wb"))

# Save validation results GARCH


# pickle.dump(GARCH_params, open("GARCH_params.pkl", "wb"))
# pickle.dump(param_grid_garch, open("param_grid_garch.pkl", "wb"))
# pickle.dump(time_to_train_GARCH, open("time_to_train_GARCH.pkl", "wb"))
# pickle.dump(all_aic, open("all_aic.pkl", "wb"))
# pickle.dump(GARCH_val_results, open("GARCH_val_results.pkl", "wb"))


In [21]:
# Save actual prices and returns classification
# pickle.dump(model_validation.binary_val, open("binary_realized.pkl", "wb"))
# pickle.dump(model_validation.ohlc_val, open("ohlc_realized.pkl", "wb"))

In [ ]:
# direction = model_validation.rolling_window(model_validation.tech_val, 
#                                         model_validation.binary_val, 
#                                         model_validation.window, 
#                                         model_validation.val_size, 
#                                         model_validation.returns_val,
#                                         val_model='RF',
#                                         RF_params=(50, 3, None),
#                                         features=model_validation.feat_val)

In [ ]:
# train_set = pd.concat([model_validation.tech[company], model_validation.features], axis=1).shift().iloc[1:window]
# train_binary = model_validation.binary_val.iloc[:window-1][company]
# assert len(train_binary) == len(train_set)
# test_set = pd.concat([model_validation.tech[company], model_validation.features], axis=1).shift().iloc[window+1:]
# test_binary = model_validation.binary_val.iloc[window:][company]
# assert len(test_set) == len(test_binary)

In [ ]:
# from sklearn.linear_model import LinearRegression
# reg = LinearRegression().fit(train_set, train_binary)
# reg.score(train_set, train_binary)

# import statsmodels.api as sm
# X_ols = sm.add_constant(train_set) # [['MACD', 'CL=F', 'EURUSD=X']]
# model_ols = sm.OLS(train_binary,X_ols)
# results_ols = model_ols.fit()



In [ ]:
# from sklearn import svm
# clf = svm.SVC()
# clf.fit(train_set, train_binary)

In [ ]:
# res = clf.predict(train_set)

In [ ]:
# res_dict_backtest_SVC = {'f1':None,
#                     'recall':None,
#                     'precision':None,
#                     'accuracy':None}
# res_dict_backtest_SVC['f1'] = metrics.f1_score(train_binary, res)
# res_dict_backtest_SVC['recall'] = metrics.recall_score(train_binary, res)
# res_dict_backtest_SVC['precision'] = metrics.precision_score(train_binary, res)
# res_dict_backtest_SVC['accuracy'] = metrics.accuracy_score(train_binary, res)
# res_dict_backtest_SVC

In [ ]:
# res_out = clf.predict(test_set)

In [ ]:
# res_dict_out_SVC = {'f1':None,
#                     'recall':None,
#                     'precision':None,
#                     'accuracy':None}
# res_dict_out_SVC['f1'] = metrics.f1_score(test_binary, res_out)
# res_dict_out_SVC['recall'] = metrics.recall_score(test_binary, res_out)
# res_dict_out_SVC['precision'] = metrics.precision_score(test_binary, res_out)
# res_dict_out_SVC['accuracy'] = metrics.accuracy_score(test_binary, res_out)
# res_dict_out_SVC

In [ ]:
# RF_add_params = []
# parameters_check = {
#     'n_estimators': [20, 40, 60, 80, 100, 120, 140],
#     'max_depth': [3, 5, 7, 9, 11, 13, 15, None],
#     'max_leaf_nodes': [3, 5, 7, 9, 11, 13, None],
# }

# for i in itertools.product(parameters_check['n_estimators'],
#                            parameters_check['max_depth'],
#                            parameters_check['max_leaf_nodes']):
    
#     RF_add_params.append(i)

In [ ]:
# clf = make_pipeline(RandomForestClassifier(random_state=42))
# cv_acc_scores = cross_val_score(clf, train_set, train_binary, cv=5, n_jobs=-1)
# print(cv_acc_scores)
# params = {
#     "randomforestclassifier__n_estimators": range(20, 140, 20),
#     "randomforestclassifier__max_depth": range(3, 14, 3),
#     "randomforestclassifier__max_leaf_nodes": range(3, 14, 3)
# }
# model = GridSearchCV(
#     clf,
#     param_grid=params,
#     cv=5,
#     n_jobs=-1,
#     verbose=0
# )

# model.fit(train_set, train_binary)
# cv_results = pd.DataFrame(model.cv_results_)
# cv_results.sort_values('rank_test_score')

In [ ]:
# company = tickers[1]

# company

In [ ]:
# train_set = pd.concat([model_validation.tech[company], model_validation.features], axis=1).iloc[1:window]
# train_binary = model_validation.returns_val.iloc[:window-1][company] / 100
# assert len(train_binary) == len(train_set)
# test_set = pd.concat([model_validation.tech[company], model_validation.features], axis=1).iloc[window:]
# test_binary = model_validation.returns_val.iloc[window-1:][company] / 100
# assert len(test_set) == len(test_binary)

In [ ]:
# model_final = RandomForestRegressor(n_estimators=40, 
#                                max_depth=3, 
#                                max_leaf_nodes=3, 
#                                random_state=42,
#                                n_jobs=-1)


# model_final.fit(train_set, train_binary)
# insample = model_final.predict(train_set)
# outofsample = model_final.predict(test_set)

In [ ]:
# print(metrics.mean_squared_error(train_binary, insample))
# print(metrics.mean_squared_error(test_binary, outofsample))

In [ ]:
# y_pred_baseline = [train_binary.mean()] * len(train_binary)
# y_pred_baseline_test = [train_binary.mean()] * len(test_binary)

In [ ]:
# print(metrics.mean_squared_error(train_binary, y_pred_baseline))
# print(metrics.mean_squared_error(test_binary, y_pred_baseline_test))

In [ ]:
# res_dict_backtest = {'f1':None,
#                     'recall':None,
#                     'precision':None,
#                     'accuracy':None}
# res_dict_backtest['f1'] = metrics.f1_score(train_binary, insample)
# res_dict_backtest['recall'] = metrics.recall_score(train_binary, insample)
# res_dict_backtest['precision'] = metrics.precision_score(train_binary, insample)
# res_dict_backtest['accuracy'] = metrics.accuracy_score(train_binary, insample)
# res_dict_backtest

In [ ]:
# res_dict_backtest

In [ ]:
# res_dict_outofsample = {'f1':None,
#                     'recall':None,
#                     'precision':None,
#                     'accuracy':None}
# res_dict_outofsample['f1'] = metrics.f1_score(test_binary, outofsample)
# res_dict_outofsample['recall'] = metrics.recall_score(test_binary, outofsample)
# res_dict_outofsample['precision'] = metrics.precision_score(test_binary, outofsample)
# res_dict_outofsample['accuracy'] = metrics.accuracy_score(test_binary, outofsample)
# res_dict_outofsample

In [ ]:
# res_dict_outofsample